In [1]:
import sqlite3
import pandas as pd

con = sqlite3.connect('data/hockey.db')

In [2]:
df = pd.read_sql('select * from players', con)
df['date'] = df['date'].apply(pd.to_datetime)
df = df.sort_values('date', ascending=True)

In [3]:
df.date.max()

Timestamp('2019-02-14 00:00:00')

In [4]:
df = df[df.date <= '2018-12-31']

In [7]:
df.shape

(318, 12)

In [5]:
df.sample(5)

,id,name,position,date,team,venue,opponent,outcome,goals,assists,shots,ice_time
67,68,John Tavares,C,2018-10-20,TOR,Home,STL,L,0,0,1,16.0
50,51,Connor McDavid,C,2018-10-16,EDM,Away,WPG,W,2,2,3,22.0
165,166,Erik Karlsson,D,2018-11-17,SJS,Home,STL,W,1,1,4,20.0
154,155,Patrice Bergeron,C,2018-11-14,BOS,Away,COL,L,0,1,3,17.0
34,35,John Tavares,C,2018-10-11,TOR,Away,DET,W,0,4,3,18.0


In [8]:
# df = df[df['name'] == 'Sidney Crosby']
df.head(5)

,id,name,position,date,team,venue,opponent,outcome,goals,assists,shots,ice_time
0,1,Alex Ovechkin,LW,2018-10-03,WSH,Home,BOS,W,1,1,5,16.0
1,2,Auston Matthews,C,2018-10-03,TOR,Home,MTL,W,2,0,3,17.0
2,3,Brent Burns,D/RW,2018-10-03,SJS,Home,ANA,L,0,0,2,25.0
3,4,Erik Karlsson,D,2018-10-03,SJS,Home,ANA,L,0,0,1,26.0
4,5,John Tavares,C,2018-10-03,TOR,Home,MTL,W,1,0,5,18.0


In [9]:
rolling = (
    df
    .groupby(['name', 'position'])
    [['goals', 'assists', 'shots', 'ice_time']]
    .rolling(5)
    .mean()
    .groupby(['name', 'position'])
    .shift(1)
    .reset_index()
    .rename(columns={'level_2': 'index'})
    .set_index('index')
)

In [10]:
train = pd.merge(df, rolling, left_index=True, right_index=True, suffixes=('', '_rolling'))

In [11]:
train.head()

,id,name,position,date,team,venue,opponent,outcome,goals,assists,shots,ice_time,name_rolling,position_rolling,goals_rolling,assists_rolling,shots_rolling,ice_time_rolling
0,1,Alex Ovechkin,LW,2018-10-03,WSH,Home,BOS,W,1,1,5,16.0,Alex Ovechkin,LW,NaN,NaN,NaN,NaN
1,2,Auston Matthews,C,2018-10-03,TOR,Home,MTL,W,2,0,3,17.0,Auston Matthews,C,NaN,NaN,NaN,NaN
2,3,Brent Burns,D/RW,2018-10-03,SJS,Home,ANA,L,0,0,2,25.0,Brent Burns,D/RW,NaN,NaN,NaN,NaN
3,4,Erik Karlsson,D,2018-10-03,SJS,Home,ANA,L,0,0,1,26.0,Erik Karlsson,D,NaN,NaN,NaN,NaN
4,5,John Tavares,C,2018-10-03,TOR,Home,MTL,W,1,0,5,18.0,John Tavares,C,NaN,NaN,NaN,NaN


In [12]:
train = train.dropna(subset=['goals_rolling'])

In [13]:
X = train[[
    'position', 
    'goals_rolling', 
    'assists_rolling',
    'shots_rolling',
    'ice_time_rolling',
]]

y = train['goals']

In [14]:
X[:5]

,position,goals_rolling,assists_rolling,shots_rolling,ice_time_rolling
41,C,1.2,0.8,4.2,18.8
42,RW,0.2,0.6,2.6,14.0
39,C,1.8,0.6,3.4,18.0
45,D/RW,0.0,0.6,2.6,24.4
46,D,0.0,0.6,2.4,26.4


In [15]:
y[:5]

41    0
42    1
39    1
45    0
46    0
Name: goals, dtype: int64

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.preprocessing import LabelBinarizer

In [17]:
mapper = DataFrameMapper([
    ('position', LabelBinarizer()),
    ('goals_rolling', None), 
    ('assists_rolling', None),
    ('shots_rolling', None), 
    ('ice_time_rolling', None),
], df_out=True)

In [20]:
Z = mapper.fit_transform(X)

In [21]:
model = LinearRegression()
model.fit(Z, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [22]:
model.score(Z, y)

0.116700796366069

In [23]:
test = pd.read_sql('select * from players', con)
test = df[df.date > '2018-12-31']

In [ ]:
df = pd.read_sql('select * from players', con)